---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [38]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA', 'SW_L', 'SW_L', 'PA','SW_H']
#count = 1
#for G in P1_Graphs:
#    print('Graph'+ str(count))
##    degrees = G.degree()
#    degree_values = sorted(set(degrees.values()))
#    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#    print (histogram)
#    print(np.sum(histogram))
#    print(nx.average_clustering(G))
#    print(nx.average_shortest_path_length(G))
#    count += 1

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))
    df1 = df.dropna(axis=0)
    df1 = df.dropna(axis=0)
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve, auc
    features1 = df1[['Department','degree', 'degree_centrality', 'closeness_centrality', 'betweenness_centrality']]
    target1 = df1['ManagementSalary']
    target1 = target1.astype(int)
    clf = RandomForestClassifier(n_estimators=80).fit(features1, target1)
    df2 = df[df['ManagementSalary'].isnull()]
    features2 = df2[['Department','degree', 'degree_centrality', 'closeness_centrality', 'betweenness_centrality']]
    
    y_score_clf = clf.predict_proba(features2)
    y_score_clf1 = [i[1] for i in y_score_clf]
    answer_series = pd.Series(y_score_clf1, index=df2.index)
    
    return answer_series

#df = pd.DataFrame(index=G.nodes())
#df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
#df['ManagementSalary']=pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
#df['degree'] = pd.Series(G.degree())
#df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
#df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
#df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))

                                    

#df1 = df.dropna(axis=0)
#df1['ManagementSalary'] = df1['ManagementSalary'].astype('int')
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import roc_curve, auc
#features = df1[['Department','degree', 'degree_centrality', 'closeness_centrality', 'betweenness_centrality']]
#target = df1['ManagementSalary']
#target = target.astype(int)
#X_train, X_test, y_train, y_test = train_test_split(features, target)
#clf = RandomForestClassifier(n_estimators=80).fit(X_train, y_train.astype('int'))
#y_score_clf = clf.predict_proba(X_test)
#y_score_clf1 = [i[1] for i in y_score_clf]
#fpr, tpr, _ = roc_curve(y_test, y_score_clf1)
#auc(fpr, tpr)
#len(roc_curve(y_test, y_score_clf1))
#y_score_clf
#df[df['ManagementSalary'].isnull()]
salary_predictions()


1       0.0625
2       1.0000
5       1.0000
8       0.1625
14      0.0625
18      0.2500
27      0.2375
30      0.3000
31      0.5625
34      0.0125
37      0.1000
40      0.1125
45      0.0625
54      0.2875
55      0.4125
60      0.1500
62      1.0000
65      0.9500
77      0.0500
79      0.1375
97      0.0000
101     0.0000
103     0.0875
108     0.1375
113     0.2500
122     0.0125
141     0.3500
142     1.0000
144     0.0000
145     0.0625
         ...  
913     0.0000
914     0.0000
915     0.0000
918     0.0000
923     0.0125
926     0.0125
931     0.0625
934     0.0000
939     0.0000
944     0.0000
945     0.0000
947     0.0000
950     0.2125
951     0.0000
953     0.0000
959     0.0000
962     0.0000
963     0.0375
968     0.0000
969     0.0000
974     0.0000
984     0.0000
987     0.0000
989     0.0000
991     0.0000
992     0.0125
994     0.0000
996     0.0000
1000    0.0000
1001    0.0125
Length: 252, dtype: float64

In [6]:
#nx.betweenness_centrality(G)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [45]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)


,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


In [49]:
#df = future_connections
#edge_list = list(df.index)
#edge_list_short = list(df_graph1.index)
#G = nx.Graph()
#G.add_edges_from(edge_list)
#G.edges()
#df = pd.DataFrame(index=edge_list)
#df['preferential_attachment'] = [i[2]for i in list(nx.preferential_attachment(G, edge_list))]
#df['common_neighbors'] = [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in edge_list]
#df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G, edge_list))]
#df['resource_allocation'] = [i[2] for i in list(nx.resource_allocation_index(G, edge_list))]
#df['future_connection'] = df_graph['Future Connection']
#df_target = df[df['Future Connection'].isnull()]
#df_train = df.dropna(axis=0)
#df

,Future Connection,preferential_attachment
"(6, 840)",0.0,880308
"(4, 197)",0.0,881790
"(620, 979)",0.0,1000998
"(519, 872)",0.0,974153
"(382, 423)",0.0,964035
"(97, 226)",1.0,923121
"(349, 905)",0.0,978072
"(429, 860)",0.0,954504
"(309, 989)",0.0,961920
"(468, 880)",0.0,957442


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [53]:
def new_connections_predictions():
    #df_target = future_connections[future_connections['Future Connection'].isnull()]
    #df_graph = future_connections.dropna(axis=0)
    #df_graph1 = df_graph[0::10] # this controls how many samples I take
    df = future_connections
    edge_list = list(df.index)
    #edge_list_short = list(df_graph1.index)
    G = nx.Graph()
    G.add_edges_from(edge_list)
    G.edges()
    df['preferential_attachment'] = [i[2]for i in list(nx.preferential_attachment(G, edge_list))]
    #df['common_neighbors'] = [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in edge_list]
    #df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G, edge_list))]
    #df['resource_allocation'] = [i[2] for i in list(nx.resource_allocation_index(G, edge_list))]
    #df['future_connection'] = df_graph['Future Connection']
    df_target = df[df['Future Connection'].isnull()]
    df_train = df.dropna(axis=0)
    X_train = df_train[['preferential_attachment', 'common_neighbors', 'jaccard_coefficient', 'resource_allocation']]
    y_train = df_train['Future Connection']
    X_test = df_target[['preferential_attachment', 'common_neighbors', 'jaccard_coefficient', 'resource_allocation']]
    y_test = df_target['Future Connection']
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import MinMaxScaler
    Scaler = MinMaxScaler()
    Scaler.fit(X_train)
    X_train_scaled = Scaler.transform(X_train)
    X_test_scaled = Scaler.transform(X_test)
    lr = LogisticRegression(C=10000, penalty='l2').fit(X_train_scaled,y_train)
    prob_array = lr.predict_proba(X_test_scaled)
    positive_prob_list = [i[1] for i in prob_array]

    
    
    return pd.Series(positive_prob_list, index = df_target.index)
new_connections_predictions()
#df_target = future_connections[future_connections.isnull()]
#df_graph = future_connections.dropna(axis=0)
#df_graph1 = df_graph[0::10] # this controls how many samples I take
#edge_list = list(df_graph.index)
#edge_list_short = list(df_graph1.index)
#G = nx.Graph()
#G.add_edges_from(edge_list)
#G.edges()
#df = pd.DataFrame(index=edge_list_short)
#df['preferential_attachment'] = [i[2]for i in list(nx.preferential_attachment(G, edge_list_short))]
#df['common_neighbors'] = [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in edge_list_short]
#df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G, edge_list_short))]
#df['resource_allocation'] = [i[2] for i in list(nx.resource_allocation_index(G, edge_list_short))]
#df['future_connection'] = df_graph1['Future Connection']


(107, 348)    0.594158
(542, 751)    0.028207
(20, 426)     0.177599
(50, 989)     0.024205
(942, 986)    0.019176
(324, 857)    0.027163
(13, 710)     0.449774
(19, 271)     0.200168
(319, 878)    0.022831
(659, 707)    0.038425
(49, 843)     0.023285
(208, 893)    0.048220
(377, 469)    0.265205
(405, 999)    0.169162
(129, 740)    0.321413
(292, 618)    0.044391
(239, 689)    0.022400
(359, 373)    0.072343
(53, 523)     0.053336
(276, 984)    0.021966
(202, 997)    0.021131
(604, 619)    0.023724
(270, 911)    0.024672
(261, 481)    0.095595
(200, 450)    0.121796
(213, 634)    0.043078
(644, 735)    0.026628
(346, 553)    0.034138
(521, 738)    0.045177
(422, 953)    0.055421
                ...   
(672, 848)    0.023736
(28, 127)     0.184336
(202, 661)    0.045385
(54, 195)     0.083302
(295, 864)    0.085802
(814, 936)    0.031063
(839, 874)    0.019176
(139, 843)    0.037672
(461, 544)    0.046810
(68, 487)     0.055452
(622, 932)    0.110268
(504, 936)    0.061627
(479, 528) 

,Future Connection
"(107, 348)",NaN
"(542, 751)",NaN
"(20, 426)",NaN
"(50, 989)",NaN
"(942, 986)",NaN
"(324, 857)",NaN
"(13, 710)",NaN
"(19, 271)",NaN
"(319, 878)",NaN
"(659, 707)",NaN


In [10]:
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.model_selection import train_test_split
#features = df[['preferential_attachment', 'common_neighbors', 'jaccard_coefficient', 'resource_allocation']]
#targets = df['future_connection']
#X_train, X_test, y_train, y_test = train_test_split(features, targets)
#clf = GradientBoostingClassifier(learning_rate = 0.001,max_depth=3, random_state = 0, n_estimators=150).fit(X_train, y_train)

In [11]:
#from sklearn.metrics import roc_curve, auc
#prob_array = clf.predict_proba(X_test)
#positive_prob_list = [i[1] for i in prob_array]
#fpr, tpr, _ = roc_curve(y_test, positive_prob_list)
#score = auc(fpr, tpr)
#score

0.77350466495954173

In [12]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators=150).fit(X_train, y_train)
#prob_array = clf.predict_proba(X_test)
#positive_prob_list = [i[1] for i in prob_array]
#fpr, tpr, _ = roc_curve(y_test, positive_prob_list)
#score = auc(fpr, tpr)
#score


0.73350491896448544

In [13]:
#from sklearn.naive_bayes import GaussianNB
#nbclf = GaussianNB().fit(X_train, y_train)
#prob_array = nbclf.predict_proba(X_test)
#positive_prob_list = [i[1] for i in prob_array]
#fpr, tpr, _ = roc_curve(y_test, positive_prob_list)
#score = auc(fpr, tpr)
#score


0.75854464692519064

In [35]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import MinMaxScaler
#Scaler = MinMaxScaler()
#Scaler.fit(X_train)
#X_train_scaled = Scaler.transform(X_train)
#X_test_scaled = Scaler.transform(X_test)
#lr = LogisticRegression(C=10000, penalty='l2').fit(X_train_scaled,y_train)
#prob_array = lr.predict_proba(X_test_scaled)
#positive_prob_list = [i[1] for i in prob_array]
#fpr, tpr, _ = roc_curve(y_test, positive_prob_list)
#score = auc(fpr, tpr)
#score


In [15]:
#from sklearn.model_selection import GridSearchCV
#clf = GradientBoostingClassifier()
#param_grid = [{}]

In [36]:

        
#edge_list[-1]  

In [ ]:
#df.join(df_graph)

In [ ]:
#sorted(edge_list)[-1]

In [ ]:
#sorted(G.edges())[-1]